In [1]:
import os
os.environ['USER_AGENT'] = 'MyApp/1.0 (Custom LangChain Application)'
import numpy as np
import pandas as pd

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, PyMuPDFLoader, PDFPlumberLoader, CSVLoader, UnstructuredCSVLoader
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma

In [3]:
open_embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
hf_embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3')

In [8]:
def get_embedding(text, embeddings):
    return embeddings.embed_query(text)

def cos_sim(A, B):
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

def return_answer_candidate(df, query, embeddings):
    query_embedding = get_embedding(query, embeddings)
    df['similarity'] = df.embedding.apply(lambda x: cos_sim(np.array(x), np.array(query_embedding)))
    top_3_doc = df.sort_values(by='similarity', ascending=False).head(3)

    return top_3_doc

In [6]:
data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '부동산 시장이 점점 더 복잡해지고 있어요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 변동했어요',
]
df = pd.DataFrame(data, columns=['text'])

In [7]:
open_df = df.copy()
open_df['embedding'] = df.apply(lambda row: get_embedding(row.text, open_embeddings), axis=1)
open_df

,text,embedding
0,주식 시장이 급등했어요,"[-0.013676190748810768, -0.03501933813095093, ..."
1,시장 물가가 올랐어요,"[-0.000218817382119596, -0.035430148243904114,..."
2,전통 시장에는 다양한 물품들을 팔아요,"[-0.009285671636462212, -0.014616581611335278,..."
3,부동산 시장이 점점 더 복잡해지고 있어요,"[-0.01740216836333275, -0.0030780902598053217,..."
4,저는 빠른 비트를 좋아해요,"[-0.037991348654031754, -0.013411780819296837,..."
5,최근 비트코인 가격이 많이 변동했어요,"[-0.017488501965999603, -0.02157001383602619, ..."


In [9]:
hf_df = df.copy()
hf_df['embedding'] = df.apply(lambda row: get_embedding(row.text, hf_embeddings), axis=1)
hf_df

,text,embedding
0,주식 시장이 급등했어요,"[-0.02831065095961094, -0.0020167904440313578,..."
1,시장 물가가 올랐어요,"[0.003978083375841379, 0.02656499296426773, -0..."
2,전통 시장에는 다양한 물품들을 팔아요,"[0.0001013833170873113, 0.0005807742127217352,..."
3,부동산 시장이 점점 더 복잡해지고 있어요,"[-0.0077268523164093494, 0.021975157782435417,..."
4,저는 빠른 비트를 좋아해요,"[-0.019412308931350708, 0.012223082594573498, ..."
5,최근 비트코인 가격이 많이 변동했어요,"[-0.026391346007585526, -0.004527552053332329,..."


In [10]:
query = '과일 값이 비싸다.'
open_sim_result = return_answer_candidate(open_df, query, open_embeddings)
open_sim_result

,text,embedding,similarity
2,전통 시장에는 다양한 물품들을 팔아요,"[-0.009285671636462212, -0.014616581611335278,...",0.824507
1,시장 물가가 올랐어요,"[-0.000218817382119596, -0.035430148243904114,...",0.814506
0,주식 시장이 급등했어요,"[-0.013676190748810768, -0.03501933813095093, ...",0.805363


In [11]:
hf_sim_result = return_answer_candidate(hf_df, query, hf_embeddings)
hf_sim_result

,text,embedding,similarity
1,시장 물가가 올랐어요,"[0.003978083375841379, 0.02656499296426773, -0...",0.647129
5,최근 비트코인 가격이 많이 변동했어요,"[-0.026391346007585526, -0.004527552053332329,...",0.614578
0,주식 시장이 급등했어요,"[-0.02831065095961094, -0.0020167904440313578,...",0.578653
